### 스타벅스 위치 정보

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import time
import folium
import collections
from collections import OrderedDict
import csv
import pandas as pd

In [2]:
chrome_driver = 'D:/Python/chromedriver.exe'
driver = webdriver.Chrome(chrome_driver)
driver.implicitly_wait(5) # 응답의 시간 지연
# url = 'https://www.starbucks.co.kr/store/getStore.do?r=24F3U9PI7Q'
url = 'https://www.starbucks.co.kr/store/store_map.do'
# chrome driver로 해당 페이지가 물리적으로 open
driver.get(url)

# 값을 담을 리스트
starbucks = []

# 열린 페이지에서 '지역 검색' 탭 클릭
search = driver.find_element_by_class_name('loca_search')
time.sleep(1)
search.click()

time.sleep(1)
# 개발자도구로 class : set_sido_cd_btn의 데이터 긁어옴
search = driver.find_elements_by_class_name('set_sido_cd_btn')

for item in search:
    item.click()
    time.sleep(1)

    # data-sidocd='01~17' 서울~세종
    if '17' == item.get_attribute('data-sidocd'):
        # 소스 가져오기
        src = driver.page_source

        # BeautifulSoup 객체로 변환
        soup = BeautifulSoup(src)
        name = soup.select('li[data-name]')
        for name_one in name:
            x = name_one['data-lat'] # 위도 저장
            y = name_one['data-long'] # 경도 저장
            z = name_one['data-name'] # 지점명 저장
            p = name_one.select_one('p').text.split('1522-3232')[0] # 번호는 모든 지점이 동일하여 crawling에서 제외
            starbucks.append({'name': z, 'address': p, 'lat': x, 'long':y}) # dict 형태로 리스트에 저장
        time.sleep(1)

        # 열린 페이지 닫기
        driver.close()
    else:
        search2 = driver.find_element_by_link_text('전체')
        search2.click()
        driver.implicitly_wait(5)
        time.sleep(1)

        src = driver.page_source

        soup = BeautifulSoup(src)
        name = soup.select('li[data-name]')
        for name_one in name:
            x = name_one['data-lat']
            y = name_one['data-long']
            z = name_one['data-name']
            p = name_one.select_one('p').text.split('1522-3232')[0]
            starbucks.append({'name': z, 'address': p, 'lat': x, 'long':y})
        time.sleep(1)

        # 다시 지역 검색 탭으로 돌아가기위한 소스
        search3 = driver.find_element_by_link_text('지역 검색')
        search3.click()
        time.sleep(1)

In [3]:
# 리스트 중복값 제거
starbucks = list(map(dict, collections.OrderedDict.fromkeys(tuple(sorted(d.items())) for d in starbucks)))

In [4]:
# 서울에 위치한 스타벅스 추출
seoul_starbucks = starbucks[:560]

In [8]:
# newline=''을 넣으면 공백라인 제거
with open('starbucks_loc_info.csv', 'w', newline='') as f:
    fieldnames =['지점명', '주소', '위도', '경도']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for i in seoul_starbucks:
        writer.writerow({'지점명': i['name'], '주소': i['address'], '위도': i['lat'], '경도': i['long']})

In [9]:
starbucks = pd.read_csv('starbucks_loc_info.csv', encoding="EUC-KR")
starbucks

,지점명,주소,위도,경도
0,애오개역,"서울특별시 마포구 마포대로 201, 마포트라팰리스II (아현동)",37.552187,126.955520
1,마포경찰서후문,서울특별시 마포구 마포대로 173-14 (아현동),37.550800,126.953432
2,아현역,"서울특별시 서대문구 신촌로35길 10 (북아현동, E편한세상신촌 4단지)",37.558421,126.956349
3,공덕,서울특별시 마포구 마포대로 130 (공덕동),37.545913,126.952956
4,이대R,서울특별시 서대문구 이화여대길 34 (대현동),37.558483,126.945964
5,서강대프라자,서울특별시 마포구 백범로 35 (신수동),37.551073,126.943096
6,마포이마트,서울특별시 마포구 백범로 212 (신공덕동) 마포이마트,37.542421,126.953341
7,마포염리,서울특별시 마포구 독막로 311 (염리동),37.543876,126.947680
8,공덕오거리,서울특별시 마포구 마포대로 86 (도화동) 창강빌딩,37.542464,126.950577
9,충정타워,"서울특별시 서대문구 서소문로 21, 충정타워빌딩 1층 (충정로3가)",37.559869,126.964110
